# Radial Basis Function Model

In [21]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from pdb import set_trace
from data.data_utils import load_dataset
from scipy.linalg import cho_factor, cho_solve

In [22]:
np.random.seed(42)

In [23]:
# MULTIDIMENSIONAL KERNEL DEFINITION
def gaussian_kernel(x, z, theta=1.):
    """
    Evaluate the Gram matrix for a Gaussian kernel between points in x and z.
    Inputs:
        x : array of shape (N, d)
        z : array of shape (M, d)
        theta : lengthscale parameter (>0)
    Outputs:
        k : Gram matrix of shape (N, M)
    """
    # reshape the matricies correctly for broadcasting
    x = np.expand_dims(x, axis=1)
    z = np.expand_dims(z, axis=0)
    # now evaluate the kernel using the euclidean distances squared between points
    return np.exp(-np.sum(np.square(x-z)/theta, axis=2, keepdims=False))

# TWO WAYS TO WRITE LEAST SQAURE ERROR LOSS FUNCTION
def lse_loss(y_true, y_pred):
    return np.linalg.norm(y_true - y_pred) ** 2 / (y_true.shape[0])

def lse_loss2(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))


# Mauna_loa

In [24]:
x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('mauna_loa')
thetaL = np.array([0.05, 0.1, 0.5, 1, 2])
lambdaL = np.array([0.001, 0.01, 0.1, 1])
# a = (K + lambd*I)^-1 y
I = np.identity(x_train.shape[0])
loss = np.empty((thetaL.shape[0], lambdaL.shape[0]))
for i, theta in enumerate(thetaL):
    for j, lambd in enumerate(lambdaL):
        # get the Cholesky decomposition of (K + lambd*I)
        K = gaussian_kernel(x_train, x_train, theta) # form Gram matrix
        C = cho_factor(K + lambd*I) # cholesky factorize
        alpha = cho_solve(C, y_train) # compute parameters
        yy = gaussian_kernel(x_valid, x_train, theta).dot(alpha) # predict
        loss[i][j] = lse_loss(y_valid, yy)
        print("theta: %f\tlamda: %f\tloss: %f" %(theta, lambd, loss[i][j]))
loss

theta: 0.050000	lamda: 0.001000	loss: 1.487690
theta: 0.050000	lamda: 0.010000	loss: 1.248379
theta: 0.050000	lamda: 0.100000	loss: 1.170763
theta: 0.050000	lamda: 1.000000	loss: 1.192925
theta: 0.100000	lamda: 0.001000	loss: 2.005867
theta: 0.100000	lamda: 0.010000	loss: 1.121770
theta: 0.100000	lamda: 0.100000	loss: 0.932977
theta: 0.100000	lamda: 1.000000	loss: 0.993464
theta: 0.500000	lamda: 0.001000	loss: 0.120479
theta: 0.500000	lamda: 0.010000	loss: 0.182945
theta: 0.500000	lamda: 0.100000	loss: 0.224430
theta: 0.500000	lamda: 1.000000	loss: 0.367641
theta: 1.000000	lamda: 0.001000	loss: 0.015495
theta: 1.000000	lamda: 0.010000	loss: 0.052668
theta: 1.000000	lamda: 0.100000	loss: 0.114997
theta: 1.000000	lamda: 1.000000	loss: 0.196794
theta: 2.000000	lamda: 0.001000	loss: 0.040685
theta: 2.000000	lamda: 0.010000	loss: 0.063708
theta: 2.000000	lamda: 0.100000	loss: 0.047152
theta: 2.000000	lamda: 1.000000	loss: 0.062111


array([[1.48768999, 1.24837923, 1.17076297, 1.19292526],
       [2.00586689, 1.12177039, 0.93297741, 0.9934635 ],
       [0.12047903, 0.18294482, 0.22443026, 0.36764121],
       [0.01549494, 0.05266788, 0.11499661, 0.19679406],
       [0.04068502, 0.06370791, 0.04715181, 0.06211053]])

In [25]:
best = np.unravel_index(np.argmin(loss), loss.shape) # find the index for smallest loss (row, col)
theta = thetaL[best[0]]
lambd = lambdaL[best[1]]
print("Best params: thet: %f\tlambda: %f\tvalid_loss: %f" %(theta, lambd, loss[best]))

Best params: thet: 1.000000	lambda: 0.001000	valid_loss: 0.015495


In [26]:
# Test data
# Merge splits
x = np.vstack([x_valid, x_train])
y = np.vstack([y_valid, y_train])
I2 = np.identity(x.shape[0])
# get the Cholesky decomposition of (K + lambd*I)
K = gaussian_kernel(x, x, theta) # form Gram matrix
C = cho_factor(K + lambd*I2) # cholesky factorize
alpha = cho_solve(C, y) # compute parameters
yy2 = gaussian_kernel(x_test, x, theta).dot(alpha) # predict
loss2 = lse_loss(y_test, yy2)
print("Best params: thet: %f\tlambda: %f\ttest_loss: %f" %(theta, lambd,loss2))

Best params: thet: 1.000000	lambda: 0.001000	test_loss: 0.022432


# Rosenbrock, n_train=1000, d=2

In [27]:
x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('rosenbrock', n_train=1000, d=2)


In [28]:
thetaL = np.array([0.05, 0.1, 0.5, 1, 2])
lambdaL = np.array([0.001, 0.01, 0.1, 1])
# a = (K + lambd*I)^-1 y
I = np.identity(x_train.shape[0])
loss = np.empty((thetaL.shape[0], lambdaL.shape[0]))
for i, theta in enumerate(thetaL):
    for j, lambd in enumerate(lambdaL):
        # get the Cholesky decomposition of (K + lambd*I)
        K = gaussian_kernel(x_train, x_train, theta) # form Gram matrix
        C = cho_factor(K + lambd*I) # cholesky factorize
        alpha = cho_solve(C, y_train) # compute parameters
        yy = gaussian_kernel(x_valid, x_train, theta).dot(alpha) # predict
        loss[i][j] = lse_loss(y_valid, yy)
        print("theta: %f\tlamda: %f\tloss: %f" %(theta, lambd, loss[i][j]))
loss

theta: 0.050000	lamda: 0.001000	loss: 0.540906
theta: 0.050000	lamda: 0.010000	loss: 0.545992
theta: 0.050000	lamda: 0.100000	loss: 0.565966
theta: 0.050000	lamda: 1.000000	loss: 0.653030
theta: 0.100000	lamda: 0.001000	loss: 0.392609
theta: 0.100000	lamda: 0.010000	loss: 0.399459
theta: 0.100000	lamda: 0.100000	loss: 0.419560
theta: 0.100000	lamda: 1.000000	loss: 0.519150
theta: 0.500000	lamda: 0.001000	loss: 0.123557
theta: 0.500000	lamda: 0.010000	loss: 0.145166
theta: 0.500000	lamda: 0.100000	loss: 0.175639
theta: 0.500000	lamda: 1.000000	loss: 0.263490
theta: 1.000000	lamda: 0.001000	loss: 0.066170
theta: 1.000000	lamda: 0.010000	loss: 0.088451
theta: 1.000000	lamda: 0.100000	loss: 0.128296
theta: 1.000000	lamda: 1.000000	loss: 0.217749
theta: 2.000000	lamda: 0.001000	loss: 0.037342
theta: 2.000000	lamda: 0.010000	loss: 0.058094
theta: 2.000000	lamda: 0.100000	loss: 0.097161
theta: 2.000000	lamda: 1.000000	loss: 0.190573


array([[0.54090609, 0.54599187, 0.5659655 , 0.65302976],
       [0.39260905, 0.39945925, 0.41956037, 0.51914981],
       [0.12355709, 0.14516644, 0.17563918, 0.26349018],
       [0.06617027, 0.08845106, 0.12829616, 0.21774862],
       [0.03734154, 0.05809418, 0.09716117, 0.19057335]])

In [29]:
best = np.unravel_index(np.argmin(loss), loss.shape) # find the index for smallest loss (row, col)
theta = thetaL[best[0]]
lambd = lambdaL[best[1]]
print("Best params: thet: %f\tlambda: %f\tvalid_loss: %f" %(theta, lambd, loss[best]))

Best params: thet: 2.000000	lambda: 0.001000	valid_loss: 0.037342


In [30]:
# Test data
# Merge splits
x = np.vstack([x_valid, x_train])
y = np.vstack([y_valid, y_train])
I2 = np.identity(x.shape[0])
# get the Cholesky decomposition of (K + lambd*I)
K = gaussian_kernel(x, x, theta) # form Gram matrix
C = cho_factor(K + lambd*I2) # cholesky factorize
alpha = cho_solve(C, y) # compute parameters
yy2 = gaussian_kernel(x_test, x, theta).dot(alpha) # predict
loss2 = lse_loss(y_test, yy2)
print("Best params: thet: %f\tlambda: %f\ttest_loss: %f" %(theta, lambd,loss2))

Best params: thet: 2.000000	lambda: 0.001000	test_loss: 0.021941
